***Note:*** If using Spark < 3.4 (current version of Mosaic is using Spark 3.2.1), version of Pandas needs to be downgraded to avoid `'DataFrame' object has no attribute 'iteritems'` error

In [ ]:
%pip install -qU pandas==1.5.3

In [ ]:
import os
import json
from importlib.metadata import version

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import mosaic as mos
import mosaic.api as api

In [ ]:
def set_up(use_jts=False):

    library_location = f"{mos.__path__[0]}/lib/mosaic-{version('databricks-mosaic')}-jar-with-dependencies.jar"
    if not os.path.exists(library_location):
        library_location = f"{mos.__path__[0]}/lib/mosaic-{version('databricks-mosaic')}-SNAPSHOT-jar-with-dependencies.jar"

    spark = (
        SparkSession.builder.master("local")
        .config("spark.driver.host", "localhost")
        .config("spark.jars", library_location)
        .getOrCreate()
    )
    spark.conf.set("spark.databricks.labs.mosaic.jar.autoattach", "false")
    spark.sparkContext.setLogLevel("warn")
    if use_jts:
        spark.conf.set("spark.databricks.labs.mosaic.geometry.api", "JTS")
    api.enable_mosaic(spark)

    return spark

In [ ]:
spark = set_up()

In [ ]:
# Create some dataframe here
df = (
    spark.createDataFrame([{'line_segment': 'LINESTRING (-83.488541 42.220045, -83.4924813 42.2198315)'}])
    .withColumn('h3_index', mos.grid_geometrykringexplode('line_segment', F.lit(13), F.lit(2)))
)
df.printSchema()
df.count()

***Note:*** `mosaic_kepler` magic requires `dbutis` to show the Kepler viewer in the same cell, otherwise it will output the results as HTML string. A small temporary modification to the `python/mosaic/utils/notebook_utils.py` file will write that HTML output to a file which can be displayed in IFrame, as shown the cell below next. Add the following code to that file just above the `print(html)`:
```
with open('./mosaic_kepler_view.html', 'w') as f:
    f.write(html)
```

In [ ]:
%%mosaic_kepler
df h3_index h3 100_000

In [ ]:
from IPython.display import IFrame

IFrame(src='./mosaic_kepler_view.html', width=1000, height=600)

In [ ]:
spark.stop()